In [24]:
import boto3
import boto3
import json
import time
import zipfile
from io import BytesIO



iam_client = boto3.client('iam')
sts_client = boto3.client('sts')
lambda_client = boto3.client('lambda')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]

suffix = f"{region}-{account_id}"
agent_name = "hr-assistant-function-def"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_description = "Agent for providing HR assistance to manage vacation time"
agent_instruction = "You are an HR agent, helping employees understand HR policies and manage vacation time"
agent_action_group_name = "VacationsActionGroup"
agent_action_group_description = "Actions for getting the number of available vacations days for an employee and confirm new time off"
agent_alias_name = f"{agent_name}-alias"
lambda_function_role = f'{agent_name}-lambda-role-{suffix}'
lambda_function_name = f'{agent_name}-{suffix}'

In [4]:
%%writefile get_available_vacations_days.py
import os
import json
import shutil
import sqlite3
from datetime import datetime


def get_available_vacations_days(employee_id):
    conn = sqlite3.connect('/tmp/employee_database.db')
    c = conn.cursor()

    if employee_id:
        c.execute("""
            SELECT employee_vacation_days_available
            FROM vacations
            WHERE employee_id = ?
            ORDER BY year DESC
            LIMIT 1
        """, (employee_id,))

        available_vacation_days = c.fetchone()

        if available_vacation_days:
            available_vacation_days = available_vacation_days[0]
            conn.close()
            return available_vacation_days
        else:
            conn.close()
            return f"No vacation data found for employed_id {employee_id}"
    else:
        conn.close()
        raise Exception("No employee id provided")

Overwriting get_available_vacations_days.py


In [27]:
%%writefile reserve_vacation_time.py

import os
import json
import shutil
import sqlite3
from datetime import datetime

def reserve_vacation_time(employee_id, start_date, end_date):
    conn = sqlite3.connect('/tmp/employee_database.db')
    c = conn.cursor()
    
    if not employee_id:
        conn.close()
        raise Exception("No employee id provided")
    
    if not start_date or not end_date:
        conn.close()
        raise Exception("Both start_date and end_date are required")
    
    try:
        # Calculate vacation days requested
        start_date_obj = datetime.strptime(start_date, '%Y-%m-%d').date()
        end_date_obj = datetime.strptime(end_date, '%Y-%m-%d').date()
        vacation_days_requested = (end_date_obj - start_date_obj).days + 1
        
        current_year = datetime.now().year
        
        # Insert the planned vacation
        c.execute("""
            INSERT INTO planned_vacations 
            (employee_id, vacation_start_date, vacation_end_date, vacation_days_taken) 
            VALUES (?, ?, ?, ?)
        """, (employee_id, start_date, end_date, vacation_days_requested))
        
        # Update vacation days
        c.execute("""
            UPDATE vacations 
            SET employee_vacation_days_taken = employee_vacation_days_taken + ?, 
                employee_vacation_days_available = employee_vacation_days_available - ?
            WHERE employee_id = ? AND year = ?
        """, (vacation_days_requested, vacation_days_requested, employee_id, current_year))
        
        conn.commit()
        conn.close()
        
        return f"Vacation reserved successfully from {start_date} to {end_date} ({vacation_days_requested} days)"
        
    except Exception as e:
        conn.rollback()
        conn.close()
        raise Exception(str(e))

Overwriting reserve_vacation_time.py


In [20]:
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_function_role,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_function_role)

iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

{'ResponseMetadata': {'RequestId': 'f3c313fb-5246-4a5c-8e18-50477a232217',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 15 Aug 2025 05:03:55 GMT',
   'x-amzn-requestid': 'f3c313fb-5246-4a5c-8e18-50477a232217',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [31]:
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("lambda_function.py")
z.write("get_available_vacations_days.py")
z.write("reserve_vacation_time.py")
z.write("employee_database.db")
z.close()
zip_content = s.getvalue()

# Update existing function instead of creating new one
try:
    lambda_function = lambda_client.update_function_code(
        FunctionName=lambda_function_name,
        ZipFile=zip_content
    )
    print("Lambda function updated successfully!")
    print(f"Function ARN: {lambda_function['FunctionArn']}")
except Exception as e:
    print(f"Error updating Lambda function: {e}")
    # If update fails, try to create new function
    try:
        lambda_function = lambda_client.create_function(
            FunctionName=lambda_function_name,
            Runtime='python3.12',
            Timeout=180,
            Role=lambda_iam_role['Role']['Arn'],
            Code={'ZipFile': zip_content},
            Handler='lambda_function.lambda_handler'
        )
        print("Lambda function created successfully!")
    except Exception as create_error:
        print(f"Error creating Lambda function: {create_error}")

Lambda function updated successfully!
Function ARN: arn:aws:lambda:us-east-1:447924746755:function:hr-assistant-function-def-us-east-1-447924746755
